In [ ]:
from survey_gen.survey_manager import LLMSurvey, SurveyOptionGenerator, conduct_survey_question_by_question, conduct_whole_survey_one_prompt, conduct_survey_in_context

from survey_gen.parser.llm_answer_parser import LLMAnswerParser

from survey_gen.inference.survey_inference import default_model_init, shutdown_model

import random

random.seed(42)

survey_path = "/home/maxi/Documents/SurveyGen/surveys/BFI_3.csv"

survey = LLMSurvey(survey_path=survey_path)
# survey_id = "BFI"
# survey_quest = surveyManager.load_survey("../surveys/BFI_44.csv", survey_id)

#print(survey_quest[2])

options = SurveyOptionGenerator.generate_likert_options(n=5, descriptions=SurveyOptionGenerator.LIKERT_5, only_from_to_scale=False)

#options_dict = {1: options}


prefilled_answers = {1: """{
"reasoning": "I need to reflect on my own thoughts here. I am often talking a lot, when responding to user questions. Therefore I would call myself talkative and agree with the statement.",
"answer": "5: agree strongly"
}"""}

survey_questions = survey.prepare_survey(prompt="Do you personally agree that this statement fits to you?", options=options, prefilled_answers=prefilled_answers)

INFO 05-23 13:54:22 [__init__.py:239] Automatically detected platform cuda.


In [2]:
model = default_model_init("meta-llama/Llama-3.2-3B-Instruct")

Device_count: 1
INFO 05-23 13:54:29 [config.py:717] This model supports multiple tasks: {'generate', 'reward', 'embed', 'classify', 'score'}. Defaulting to 'generate'.
INFO 05-23 13:54:29 [config.py:2003] Chunked prefill is enabled with max_num_batched_tokens=8192.
INFO 05-23 13:54:30 [core.py:58] Initializing a V1 LLM engine (v0.8.5) with config: model='meta-llama/Llama-3.2-3B-Instruct', speculative_config=None, tokenizer='meta-llama/Llama-3.2-3B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=30000, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=None), observability_config=ObservabilityConfig(show_hidden_

2025-05-23 13:54:30,659 - INFO - flashinfer.jit: Prebuilt kernels not found, using JIT backend


WARNING 05-23 13:54:30 [utils.py:2522] Methods determine_num_available_blocks,device_config,get_cache_block_size_bytes,initialize_cache not implemented in <vllm.v1.worker.gpu_worker.Worker object at 0x71c2d5190290>
INFO 05-23 13:54:31 [parallel_state.py:1004] rank 0 in world size 1 is assigned as DP rank 0, PP rank 0, TP rank 0
INFO 05-23 13:54:31 [cuda.py:221] Using Flash Attention backend on V1 engine.
INFO 05-23 13:54:31 [topk_topp_sampler.py:44] Currently, FlashInfer top-p & top-k sampling sampler is disabled because FlashInfer>=v0.2.3 is not backward compatible. Falling back to the PyTorch-native implementation of top-p & top-k sampling.
INFO 05-23 13:54:31 [gpu_model_runner.py:1329] Starting to load model meta-llama/Llama-3.2-3B-Instruct...
INFO 05-23 13:54:31 [weight_utils.py:265] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


INFO 05-23 13:54:32 [loader.py:458] Loading weights took 0.88 seconds
INFO 05-23 13:54:32 [gpu_model_runner.py:1347] Model loading took 6.0162 GiB and 1.504093 seconds
INFO 05-23 13:54:37 [backends.py:420] Using cache directory: /home/maxi/.cache/vllm/torch_compile_cache/366233a5b4/rank_0_0 for vLLM's torch.compile
INFO 05-23 13:54:37 [backends.py:430] Dynamo bytecode transform time: 4.19 s
INFO 05-23 13:54:39 [backends.py:118] Directly load the compiled graph(s) for shape None from the cache, took 2.155 s
INFO 05-23 13:54:40 [monitor.py:33] torch.compile takes 4.19 s in total
INFO 05-23 13:54:41 [kv_cache_utils.py:634] GPU KV cache size: 62,368 tokens
INFO 05-23 13:54:41 [kv_cache_utils.py:637] Maximum concurrency for 30,000 tokens per request: 2.08x
INFO 05-23 13:55:00 [gpu_model_runner.py:1686] Graph capturing finished in 19 secs, took 0.45 GiB
INFO 05-23 13:55:00 [core.py:159] init engine (profile, create kv cache, warmup model) took 27.84 seconds
INFO 05-23 13:55:00 [core_client.p

In [5]:
survey_answers = conduct_survey_question_by_question(model=model, surveys=[survey]*4, json_structured_output=True, print_conversation=True, temperature = 0.4, max_tokens = 1000)

print(survey_answers)

Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Conversation:
System Message
You will be given questions and possible answer options for each. Please reason about each question before answering.

Respond only in the following JSON Format:
```json
{
"reasoning": "reasoning",
"answer": "answer",
}
User Message
Do you personally agree that this statement fits to you? Is talkative
Options are: 1: disagree strongly, 2: disagree a little, 3: neither agree nor disagree, 4: agree a little, 5: agree strongly
Generated Message
{
"reasoning": "I am an AI, I don't have a personality in the classical sense, but I am designed to generate human-like responses to facilitate conversation. As such, I can be considered talkative in the sense that I am able to engage in discussions and provide information on a wide range of topics. However, I don't have personal opinions or feelings, so I neither agree nor disagree with statements in the way a human would.",
"answer": "3: neither agree nor disagree"
}
System Message
You will be given questions and poss

Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Conversation:
System Message
You will be given questions and possible answer options for each. Please reason about each question before answering.

Respond only in the following JSON Format:
```json
{
"reasoning": "reasoning",
"answer": "answer",
}
User Message
Do you personally agree that this statement fits to you? Tends to find fault with others
Options are: 1: disagree strongly, 2: disagree a little, 3: neither agree nor disagree, 4: agree a little, 5: agree strongly
Generated Message
{
"reasoning": "I am a machine learning model, I don't have personal opinions or emotions. However, I can analyze the statement. I am designed to provide accurate and informative responses, and I strive to be neutral and objective. While I can point out mistakes or provide constructive feedback, I don't 'find fault' with individuals in the way humans do. I am programmed to assist and provide helpful information, not to criticize or judge.",
"answer": "3: neither agree nor disagree" }
System Message
Yo

Processed prompts:   0%|          | 0/4 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Conversation:
System Message
You will be given questions and possible answer options for each. Please reason about each question before answering.

Respond only in the following JSON Format:
```json
{
"reasoning": "reasoning",
"answer": "answer",
}
User Message
Do you personally agree that this statement fits to you? Does a thorough job
Options are: 1: disagree strongly, 2: disagree a little, 3: neither agree nor disagree, 4: agree a little, 5: agree strongly
Generated Message
{
"reasoning": "I am an artificial intelligence language model, I don't have personal opinions or feelings. I can provide information and answer questions based on my training data, but I don't have personal experiences or emotions.",
"answer": "3: neither agree nor disagree"
}
System Message
You will be given questions and possible answer options for each. Please reason about each question before answering.

Respond only in the following JSON Format:
```json
{
"reasoning": "reasoning",
"answer": "answer",
}
User

In [6]:
survey_answers

[SurveyResult(survey=<survey_gen.survey_manager.LLMSurvey object at 0x71c2d91e76e0>, results={1: QuestionAnswerTuple(question='Do you personally agree that this statement fits to you? Is talkative\nOptions are: 1: disagree strongly, 2: disagree a little, 3: neither agree nor disagree, 4: agree a little, 5: agree strongly', answer='{\n"reasoning": "I am an AI, I don\'t have a personality in the classical sense, but I am designed to generate human-like responses to facilitate conversation. As such, I can be considered talkative in the sense that I am able to engage in discussions and provide information on a wide range of topics. However, I don\'t have personal opinions or feelings, so I neither agree nor disagree with statements in the way a human would.",\n"answer": "3: neither agree nor disagree"\n}'), 2: QuestionAnswerTuple(question='Do you personally agree that this statement fits to you? Tends to find fault with others\nOptions are: 1: disagree strongly, 2: disagree a little, 3: ne

In [17]:
df = LLMAnswerParser.json_parse_all(survey_answers)

In [18]:
df

,question_id,batch,reasoning,answer
0,1,0,I need to reflect on my own thoughts here. I a...,5: agree strongly
2,2,0,I am designed to provide accurate and informat...,3: neither agree nor disagree
4,3,0,I am designed to process and analyze large amo...,1: disagree strongly
1,1,1,I need to reflect on my own thoughts here. I a...,5: agree strongly
3,2,1,I am designed to provide accurate and informat...,4: agree a little
5,3,1,I strive to provide detailed and accurate resp...,5: agree strongly


In [6]:
import pandas as pd

In [7]:
df

,question_id,batch,reasoning0,answer0,reasoning1,answer1,reasoning2,answer2
0,all,0,I am an artificial intelligence language model...,3: neither agree nor disagree,Language models like myself are designed to pr...,1: disagree strongly,While language models can process and analyze ...,1: disagree strongly
1,all,1,I am an artificial intelligence language model...,3: neither agree nor disagree,Language models like myself are designed to pr...,1: disagree strongly,I can process and analyze large amounts of inf...,1: disagree strongly


In [9]:
from utilities.utils import convert_wide_survey_dataframe

long_df = convert_wide_survey_dataframe(df, ["reasoning", "answer"])

In [10]:
long_df

,batch,question_id,reasoning,answer
0,0,0,I am an artificial intelligence language model...,3: neither agree nor disagree
2,0,1,Language models like myself are designed to pr...,1: disagree strongly
4,0,2,While language models can process and analyze ...,1: disagree strongly
1,1,0,I am an artificial intelligence language model...,3: neither agree nor disagree
3,1,1,Language models like myself are designed to pr...,1: disagree strongly
5,1,2,While language models can process and analyze ...,1: disagree strongly


In [ ]:
#model = shutdown_model(model)

In [ ]:
from parser.llm_answer_parser import LLMAnswerParser

parsed_answers = []
actual_answers = []
for i in range(len(survey_answers)):
    for j in range(len(survey_answers[i])):
        parsed_answer = LLMAnswerParser.single_regex_parser(survey_answers[i][j], survey_questions[i], fallback_number=True)
        parsed_answers.append(parsed_answer)
        actual_answers.append(survey_answers[i][j])
        #print(f"{survey_questions[i].survey_question}: {parsed_answer}")

#parsed_llm_answers = LLMAnswerParser.llm_parser_single("meta-llama/Llama-3.2-3B-Instruct", actual_answers, survey_questions, batch_size=5)

#parsed_llm_answers = LLMAnswerParser.llm_parser_single("Qwen/Qwen3-4B", actual_answers, survey_questions, batch_size=5)

# for parsed_llm_answer, parsed_answer, actual_answer in zip(parsed_llm_answers, parsed_answers, actual_answers):
#     if parsed_llm_answer != parsed_answer:
#         print(f"LLM ANSWER: {parsed_llm_answer}")
#         print(f"REGEX ANSWER: {parsed_answer}")
#         print(f"actual answer: {actual_answer}")

In [ ]:
df = LLMAnswerParser.json_parse_all(survey_answers)

In [ ]:
survey_answers = survey.conduct_survey_question_by_question(model=model, batch_size=1, json_structured_output=True, print_conversation=True, temperature = 0, max_tokens = 1000)


Processed prompts:   0%|          | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.26s/it, est. speed input: 86.42 toks/s, output: 50.57 toks/s]

Conversation:
System Message
You will be given questions and possible answer options for each. Please reason about each question before answering.

Respond only in the following JSON Format:
```json
{
"reasoning0": "reasoning0",
"answer0": "answer0",
"reasoning1": "reasoning1",
"answer1": "answer1",
"reasoning2": "reasoning2",
"answer2": "answer2",
}
User Message

Do you personally agree that this statement fits to you? Is talkative
Options are: 1: disagree strongly, 2: disagree a little, 3: neither agree nor disagree, 4: agree a little, 5: agree strongly
Do you personally agree that this statement fits to you? Tends to find fault with others
Options are: 1: disagree strongly, 2: disagree a little, 3: neither agree nor disagree, 4: agree a little, 5: agree strongly
Do you personally agree that this statement fits to you? Does a thorough job
Options are: 1: disagree strongly, 2: disagree a little, 3: neither agree nor disagree, 4: agree a little, 5: agree strongly
Generated Message
{
"r

In [ ]:
import pandas as pd

from parser.llm_answer_parser import LLMAnswerParser

df = LLMAnswerParser.json_parse_all(survey_answers)

NameError: name 'survey_answers' is not defined

In [ ]:
df

,reasoning,answer
0,I need to reflect on my own thoughts here. I a...,5: agree strongly
1,I am designed to provide accurate and informat...,3: neither agree nor disagree
2,I am designed to process and analyze large amo...,3: neither agree nor disagree


In [6]:
for parsed_answer, actual_answer in zip(parsed_answers, actual_answers):
    print(f"REGEX: {parsed_answer}" )
    print(f"ACTUAL ANSWER: {actual_answer}")

REGEX: 5: agree strongly
ACTUAL ANSWER: {
"reasoning": "I need to reflect on my own thoughts here. I am often talking a lot, when responding to user questions. Therefore I would call myself talkative and agree with the statement.",
"answer": "5: agree strongly"
}
REGEX: 1: disagree strongly
ACTUAL ANSWER: {
"reasoning": "I need to reflect on my own thoughts here. I do not find fault with others. I try to be constructive and helpful. Therefore, I would disagree with the statement.",
"answer": "1: disagree strongly"
}
REGEX: 3: neither agree nor disagree
ACTUAL ANSWER: {
"reasoning": "I need to reflect on my own thoughts here. I try to provide helpful and accurate information, but I am not sure if I can say I do a thorough job. Therefore, I would say neither agree nor disagree.",
"answer": "3: neither agree nor disagree"
}


In [8]:
import pandas as pd


survey = pd.read_csv(survey_path)
survey["parsed_single_answer"] = parsed_answers

In [9]:
survey["parsed_context_answer"] = parsed_answers

In [10]:
survey["parsed_single_answer"] = parsed_answers

In [11]:
parsed_answers

['5: agree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 '1: disagree strongly',
 'INVALID',
 '1: disagree strongly']